In [1]:
import os
# Set spark environments
os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from delta import *

conf = SparkConf() 

builder = SparkSession.builder.config(conf=conf) \
    .master("spark://spark-master:7077") \
    .appName("JupyterSpark")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5b613677-e9a3-4ce9-b05b-8184a29c5e04;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 225ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

In [2]:
# Creating a simple DataFrame
df = spark.createDataFrame([(1, 'John'), (2, 'Mike'), (3, 'Sara')], ['id', 'name'])
df.write.format("delta").mode("overwrite").save("s3a://mlflow-dwh/test_delta")

### Structured streaming from file to Kafka topic
#### Note MinIO

In [8]:
ds = spark.readStream.format('delta').load("s3a://mlflow-dwh/test_delta").selectExpr("CAST(id AS STRING) as key", "CAST(name AS STRING) as value") \
  .writeStream.format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "new-fancy-topic") \
  .option("checkpointLocation", "s3a://mlflow-dwh/tmp/delta_checkpoints") \
  .start()

23/10/18 12:13:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [2]:
df = (spark.readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka:9092")
  .option("subscribe", "numtest")
  .option("startingOffsets", "latest")
  .load()
)

In [3]:
from pyspark.sql.types import  StructType, StructField, IntegerType
from pyspark.sql.functions import from_json, col
schema = StructType([
        StructField("number", IntegerType()),
])
value_df = df.select(from_json(col("value").cast("string"),schema).alias("value"))

In [4]:
exploded_df = value_df.selectExpr('value.number as number')

In [5]:
query = (exploded_df.writeStream
         .outputMode("append") # or "complete" or "update" depending on your use-case
         .format("console")
         .option("truncate", "false")  # to prevent truncation of displayed data in the console
         .start()
)

query.awaitTermination() 

23/10/18 12:07:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b5ea8c2c-9503-493b-aae2-d0a426ae796d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/10/18 12:07:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/10/18 12:07:28 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+
|number|
+------+
+------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------+
|number|
+------+
|20    |
|21    |
|22    |
|23    |
|24    |
|25    |
|26    |
|27    |
|28    |
|29    |
+------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### Check http://localhost:9001/browser/mlflow-dwh for new table